In [1]:
import os 
import glob 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import cv2
from PIL import Image 

import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.utils import make_grid, save_image


from cnn_model.Vgg import VGG
from cnn_model.alexnet import AlexNet
from Grad_CAM.img_norm import Normalize, transform_img
from Grad_CAM.model_value import mv

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class_label = {'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

label_list = list(class_label.keys())

img_pth = glob.glob('./car_data/test_data/*.jpg')
pt_pth = './pt/car_front&side_pt/vgg16_cifar10_30epoch.pt'
save_pth = './result/grad_cam_image/True_result/'

label_idx = 1



In [3]:

vgg_net = VGG(3,10,'vgg16',input_size=32).to(DEVICE)
net = torch.load(pt_pth)
print(net)

print(vgg_net._modules.keys())

VGG(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequen

In [4]:
def norm_preprocess(img_pth,means,stds): # img_pth 주면 img -> torch tensor type (1,3,224,224) 
    # means = [r,g,b] / std = [r,g,b]
    p_img = Image.open(img_pth) 
    tensor_img = torch.from_numpy(np.asarray(p_img)).permute(2,0,1).unsqueeze(0).float().div(255).cuda()
    # vgg -> 1,3,224,224
    # alex -> 1,3,227,227

    tensor_img = F.interpolate(tensor_img,size=(32,32),mode='bilinear', align_corners=False)
 


    norm_mean_std = Normalize(mean=means, std=stds)
    nt_img = norm_mean_std(tensor_img)

    return nt_img , tensor_img

In [7]:
for i in img_pth:
    name = i.split('/')[2].split('\\')[1]
    img,origin_img = norm_preprocess(i,means=[0.485, 0.456, 0.406], stds=[0.229, 0.224, 0.225])
    
    grad_cam = mv('conv5',net,img,origin_img,label_idx,name,save_pth,label_list)
    best_class,best_clas_name = grad_cam.make_img()
    print(img.shape, name)
    if label_idx != best_class: 
        save_path = './result/grad_cam_image/False_result'
        print('-------'*3)
        print(i)
        print(best_class)
        print(best_clas_name)
        grads_cam = mv('features',net,img,origin_img,best_class,name,save_path,label_list,re=True)
        a,b = grads_cam.make_img()


c:\Users\mjy22\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\module.py:1053: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


torch.Size([1, 3, 32, 32]) 000099.jpg
torch.Size([1, 3, 32, 32]) 000100.jpg
torch.Size([1, 3, 32, 32]) 000101.jpg
torch.Size([1, 3, 32, 32]) 000141.jpg
torch.Size([1, 3, 32, 32]) 000143.jpg
torch.Size([1, 3, 32, 32]) 000152.jpg
torch.Size([1, 3, 32, 32]) 000153.jpg
torch.Size([1, 3, 32, 32]) 000154.jpg
torch.Size([1, 3, 32, 32]) 000191.jpg
torch.Size([1, 3, 32, 32]) 000194.jpg
torch.Size([1, 3, 32, 32]) 000417.jpg
torch.Size([1, 3, 32, 32]) 000461.jpg
torch.Size([1, 3, 32, 32]) 000716.jpg
torch.Size([1, 3, 32, 32]) 000788.jpg
torch.Size([1, 3, 32, 32]) 000851.jpg
torch.Size([1, 3, 32, 32]) 001052.jpg
torch.Size([1, 3, 32, 32]) 001097.jpg
torch.Size([1, 3, 32, 32]) 001221.jpg
torch.Size([1, 3, 32, 32]) 001273.jpg
torch.Size([1, 3, 32, 32]) 001688.jpg
torch.Size([1, 3, 32, 32]) 001883.jpg
torch.Size([1, 3, 32, 32]) 001958.jpg
torch.Size([1, 3, 32, 32]) 002001.jpg
torch.Size([1, 3, 32, 32]) 002211.jpg
